In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv
/kaggle/input/amex-data-pckl-files/train_agg.pkl
/kaggle/input/amex-data-pckl-files/test_agg.pkl


In [2]:
!pip3 install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 258.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 702.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.3/296.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 744.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
train_df = pd.read_pickle('../input/amex-data-pckl-files/train_agg.pkl', compression='gzip')
train_labels = pd.read_csv('../input/amex-default-prediction/train_labels.csv')

In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

In [5]:
train_labels = train_labels.sort_values(by=['customer_ID']).drop('customer_ID', axis=1)
train_df = pd.concat([train_df, train_labels], axis=1)

In [6]:
gc.collect()

21

In [7]:
# COMPETITION METRIC FROM Konstantin Yakovlev
# https://www.kaggle.com/kyakovlev
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/327534
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [8]:
from autogluon.core.metrics import make_scorer
amex_scorer = make_scorer(name='amex_metric',
                          score_func=amex_metric_mod,
                          optimum=1,
                           greater_is_better=True)

In [9]:


hyperparameters = {  # hyperparameters of each model type
                   'GBM': {},
                   'NN_TORCH': {},  # NOTE: comment this line out if you get errors on Mac OSX
                  }

In [10]:
gc.collect()

63

In [11]:
train_df.shape

(458913, 743)

In [12]:
# for i in range(5):
#     if i != 4:
#         temp_df = train_df.sample(n=91782, random_state=42)
#     else:
#         temp_df = train_df
#     print(temp_df.shape)
#     print(temp_df['target'].value_counts())
#     train_df = train_df.drop(temp_df.index, axis=0)
#     # break

In [13]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label='target', path=save_path, eval_metric='recall').fit(train_df.drop('customer_ID', axis=1), 
                                                                                          hyperparameters=hyperparameters, 
                                                                                          auto_stack=True,
                                                                                           num_stack_levels=0
                                                                                          # time_limit=10*60,
                                                                                          #ag_args_fit={'num_gpus': 1}
                                                                                      )

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.4.2
Python Version:     3.7.12
Operating System:   Linux
Train Data Rows:    458913
Train Data Columns: 741
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1454

[1000]	valid_set's binary_logloss: 0.229236	valid_set's recall: 0.804766
[1000]	valid_set's binary_logloss: 0.223731	valid_set's recall: 0.810893


	0.8055	 = Validation score   (recall)
	4024.73s	 = Training   runtime
	16.58s	 = Validation runtime
Fitting model: NeuralNetTorch_BAG_L1 ...
	Memory not enough to fit TabularNeuralNetTorchModel folds in parallel. Will do sequential fitting instead. 	Consider decreasing folds trained in parallel by passing num_folds_parallel to ag_args_ensemble when calling predictor.fit
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	0.8223	 = Validation score   (recall)
	6505.31s	 = Training   runtime
	44.31s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.8223	 = Validation score   (recall)
	67.82s	 = Training   runtime
	0.95s	 = Validation runtime
AutoGluon training complete, total runtime = 10739.25s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("agModels-predictClass/")
